In [1]:
import os
os.environ['HF_ENDPOINT']='hf-mirror.com'
# cmd:set HF_ENDPOINT=https://hf-mirror.com
import torch
import sys
import time
import warnings
warnings.filterwarnings("ignore")
#清理内存
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
if hasattr(torch.cuda, 'empty_cache'):
    torch.cuda.empty_cache()

In [2]:
from faster_whisper import WhisperModel
# model_size = "small"
# model_size = "medium"
#前两个很糟糕
model_size='large-v3'
download_root=r'E:\LargeModel\Speech_to_Text\fast_whisper'
# download_root = r'E:\LargeModel\Speech_to_Text\fast_whisper\models--Systran--faster-whisper-large-v3\snapshots'
model = WhisperModel(download_root+'\\'+model_size, device="cuda", compute_type="int8_float16", download_root=download_root, local_files_only=False)

In [3]:
## 在输入音频的文件位置生成一个同名的字幕文件
input_audio = r"D:\Desktop\vocal_raion_final_travel.wav"

In [4]:
def time_convert(tsecond):
    '''
    将秒转为时:分:秒,毫秒的形式
    '''
    ms = tsecond-int(tsecond)
    tsecond = int(tsecond)
    hour = tsecond//3600
    minute = tsecond%3600//60
    second = tsecond%60
    return(str(hour).rjust(2,'0')+':'+str(minute).rjust(2,'0')+':'+str(second).rjust(2,'0')+','+str(round(ms*1000)))

In [5]:
#jupyter死机的原因是D:\Anaconda\Library 中有一个libiomp5md.dl，与其他地方的同一个文件冲突
initial_prompt=None
segments, info = model.transcribe(input_audio, initial_prompt=initial_prompt, word_timestamps=True)
# segments = ','.join([segment.text for segment in segments])
num=1
# 在输入音频的文件位置生成一个同名的字幕文件
srt_file = '.'.join(input_audio.split('.')[:-1])+'.srt'

sys.stdout = open(srt_file, 'w')
for segment in segments:
        # print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
        print("%d\n%s --> %s\n%s" % (num, time_convert(segment.start), time_convert(segment.end), segment.text))
        num +=1